In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import Akima1DInterpolator
import collections
#import http.client
import scipy

Working Time table data

In [2]:
wtt = pd.ExcelFile("Current WTT wo traffic allowances.xlsx")
#print(realdata1.sheet_names)
wtt_d = wtt.parse("WTT wo Traffic Allow DN",skiprows=1)
wtt_d = wtt_d.replace('$', np.nan)
wtt_d = wtt_d.replace(to_replace=0, value=np.nan)
wtt_u = wtt.parse("WTT wo Traffic Allow UP",skiprows=1)
wtt_u = wtt_u.replace('$', np.nan)
wtt_u = wtt_u.replace(to_replace=0, value=np.nan)

Stoppage information from working timetable

In [3]:
wtt_type = pd.ExcelFile("WTT Stoppages.xlsx")
wtt_type_d = wtt_type.parse("WTT DN Stops",skiprows=0)
for i in (np.arange(0,54)):
    for j in (np.arange(3,61)):
        a = wtt_type_d.iloc[i,j] 
        a = a.replace('"', '')
        wtt_type_d.iloc[i,j] = a
wtt_type_u = wtt_type.parse("WTT UP Stops",skiprows=0)
for i in (np.arange(0,54)):
    for j in (np.arange(3,61)):
        a = wtt_type_u.iloc[i,j] 
        a = a.replace('"', '')
        wtt_type_u.iloc[i,j] = a

Computed Runtime by TCS

In [4]:
crt = pd.ExcelFile("Computed RT ver 2.xlsx")
crt_dn_00 = crt.parse("RunTime DN 00",skiprows=0)
crt_dn_00 = crt_dn_00.replace('$', np.nan)
crt_dn_00 = crt_dn_00.replace(to_replace=0, value=np.nan)
crt_dn_01 = crt.parse("RunTime DN 01",skiprows=0)
crt_dn_01 = crt_dn_01.replace('$', np.nan)
crt_dn_01 = crt_dn_01.replace(to_replace=0, value=np.nan)
crt_dn_10 = crt.parse("RunTime DN 10",skiprows=0)
crt_dn_10 = crt_dn_10.replace('$', np.nan)
crt_dn_10 = crt_dn_10.replace(to_replace=0, value=np.nan)
crt_dn_11 = crt.parse("RunTime DN 11",skiprows=0)
crt_dn_11 = crt_dn_11.replace('$', np.nan)
crt_dn_11 = crt_dn_11.replace(to_replace=0, value=np.nan)
crt_up_00 = crt.parse("RunTime UP 00",skiprows=0)
crt_up_00 = crt_dn_00.replace('$', np.nan)
crt_up_00 = crt_dn_00.replace(to_replace=0, value=np.nan)
crt_up_01 = crt.parse("RunTime UP 01",skiprows=0)
crt_up_01 = crt_dn_01.replace('$', np.nan)
crt_up_01 = crt_dn_01.replace(to_replace=0, value=np.nan)
crt_up_10 = crt.parse("RunTime UP 10",skiprows=0)
crt_up_10 = crt_dn_10.replace('$', np.nan)
crt_up_10 = crt_dn_10.replace(to_replace=0, value=np.nan)
crt_up_11 = crt.parse("RunTime UP 11",skiprows=0)
crt_up_11 = crt_dn_11.replace('$', np.nan)
crt_up_11 = crt_dn_11.replace(to_replace=0, value=np.nan)

Block section names

In [5]:
column = list(wtt_d.columns.values)
column1 = column[3:]

Actual Realized Runtime Divided by halt type

In [6]:
rt_00 = pd.read_csv("RT 00.csv",skiprows=1)
rt_00 = rt_00.replace('$', np.nan)
rt_00 = rt_00.replace(to_replace=0, value=np.nan)
rt_00[column1] = rt_00[column1].convert_objects(convert_numeric=True)
rt_01 = pd.read_csv("RT 01.csv",skiprows=1)
rt_01 = rt_01.replace('$', np.nan)
rt_01 = rt_01.replace(to_replace=0, value=np.nan)
rt_01[column1] = rt_01[column1].convert_objects(convert_numeric=True)
rt_10 = pd.read_csv("RT 10.csv",skiprows=1)
rt_10 = rt_10.replace('$', np.nan)
rt_10 = rt_10.replace(to_replace=0, value=np.nan)
rt_10[column1] = rt_10[column1].convert_objects(convert_numeric=True)
rt_11 = pd.read_csv("RT 11.csv",skiprows=1)
rt_11 = rt_11.replace('$', np.nan)
rt_11 = rt_11.replace(to_replace=0, value=np.nan)
rt_11[column1] = rt_11[column1].convert_objects(convert_numeric=True)

/home/sufiyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.
/home/sufiyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
/home/sufiyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] 

Actual Realized Runtime Divided by halt type and direction

In [7]:
rt_00_up = rt_00[rt_00['Direction']=='U']
rt_01_up = rt_01[rt_01['Direction']=='U']
rt_10_up = rt_10[rt_10['Direction']=='U']
rt_11_up = rt_11[rt_11['Direction']=='U']
rt_00_dn = rt_00[rt_00['Direction']=='D']
rt_01_dn = rt_01[rt_01['Direction']=='D']
rt_10_dn = rt_10[rt_10['Direction']=='D']
rt_11_dn = rt_11[rt_11['Direction']=='D']
#data = data[data['Speed Class']=='RAJ']

In [8]:
rt_names = [rt_00_up, rt_01_up, rt_10_up, rt_11_up, rt_00_dn, rt_01_dn, rt_10_dn, rt_11_dn]

In [9]:
def outlier(ys):
    ys = ys[~ys.isnull()]
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return (lower_bound,upper_bound)

In [10]:
l_00_d = 0 #lower quantile limit in down direction
l_01_d = 0
l_10_d = 0
l_11_d = 0
l_00_u = 0
l_01_u = 0
l_10_u = 0
l_11_u = 0
u_00_d = 0 #upper quantile limit in up direction
u_01_d = 0
u_10_d = 0
u_11_d = 0
u_00_u = 0
u_01_u = 0
u_10_u = 0
u_11_u = 0

In [11]:
def remove_outlier(lower_limit,upper_limit,train_no,block_no,direction,halt_type):
    if (direction=='D'):
        if (halt_type=='00'):
            for i in range(len(rt_00_dn.index)):
                if (rt_00_dn.iloc[i,0]==wtt_d.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_00_dn.iloc[i,block_no]>upper_limit or rt_00_dn.iloc[i,block_no]<lower_limit):
                            rt_00_dn.iloc[i,block_no]=np.nan
                    else:
                        rt_00_dn.iloc[i,block_no]=np.nan
        if (halt_type=='01'):
            for i in range(len(rt_01_dn.index)):
                if (rt_01_dn.iloc[i,0]==wtt_d.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_01_dn.iloc[i,block_no]>upper_limit or rt_01_dn.iloc[i,block_no]<lower_limit):
                            rt_01_dn.iloc[i,block_no]=np.nan
                    else:
                        rt_01_dn.iloc[i,block_no]=np.nan
        if (halt_type=='10'):
            for i in range(len(rt_10_dn.index)):
                if (rt_10_dn.iloc[i,0]==wtt_d.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_10_dn.iloc[i,block_no]>upper_limit or rt_10_dn.iloc[i,block_no]<lower_limit):
                            rt_10_dn.iloc[i,block_no]=np.nan
                    else:
                        rt_10_dn.iloc[i,block_no]=np.nan
        if (halt_type=='11'):
            for i in range(len(rt_11_dn.index)):
                if (rt_11_dn.iloc[i,0]==wtt_d.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_11_dn.iloc[i,block_no]>upper_limit or rt_11_dn.iloc[i,block_no]<lower_limit):
                            rt_11_dn.iloc[i,block_no]=np.nan
                    else:
                        rt_11_dn.iloc[i,block_no]=np.nan
    if (direction=='U'):
        if (halt_type=='00'):
            for i in range(len(rt_00_up.index)):
                if (rt_00_up.iloc[i,0]==wtt_u.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_00_up.iloc[i,block_no]>upper_limit or rt_00_up.iloc[i,block_no]<lower_limit):
                            rt_00_up.iloc[i,block_no]=np.nan
                    else:
                        rt_00_up.iloc[i,block_no]=np.nan
        if (halt_type=='01'):
            for i in range(len(rt_01_up.index)):
                if (rt_01_up.iloc[i,0]==wtt_u.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_01_up.iloc[i,block_no]>upper_limit or rt_01_up.iloc[i,block_no]<lower_limit):
                            rt_01_up.iloc[i,block_no]=np.nan
                    else:
                        rt_01_up.iloc[i,block_no]=np.nan
        if (halt_type=='10'):
            for i in range(len(rt_10_up.index)):
                if (rt_10_up.iloc[i,0]==wtt_u.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_10_up.iloc[i,block_no]>upper_limit or rt_10_up.iloc[i,block_no]<lower_limit):
                            rt_10_up.iloc[i,block_no]=np.nan
                    else:
                        rt_10_up.iloc[i,block_no]=np.nan
        if (halt_type=='11'):
            for i in range(len(rt_11_up.index)):
                if (rt_11_up.iloc[i,0]==wtt_u.iloc[train_no,0]):
                    if (lower_limit!=np.nan):
                        if (rt_11_up.iloc[i,block_no]>upper_limit or rt_11_up.iloc[i,block_no]<lower_limit):
                            rt_11_up.iloc[i,block_no]=np.nan
                    else:
                        rt_11_up.iloc[i,block_no]=np.nan       
    

In [12]:

for i in (np.arange(0,54)): #(np.arange(0,54)):    TRAIN NUMBER HELD CONSTANT
    y1_00_d = rt_00[rt_00['TrainMaskNo']==wtt_d.iloc[i,0]]
    y1_01_d = rt_01[rt_01['TrainMaskNo']==wtt_d.iloc[i,0]]
    y1_10_d = rt_10[rt_10['TrainMaskNo']==wtt_d.iloc[i,0]]
    y1_11_d = rt_11[rt_11['TrainMaskNo']==wtt_d.iloc[i,0]]
    y1_00_u = rt_00[rt_00['TrainMaskNo']==wtt_u.iloc[i,0]]
    y1_01_u = rt_01[rt_01['TrainMaskNo']==wtt_u.iloc[i,0]]
    y1_10_u = rt_10[rt_10['TrainMaskNo']==wtt_u.iloc[i,0]]
    y1_11_u = rt_11[rt_11['TrainMaskNo']==wtt_u.iloc[i,0]]
    for j in (np.arange(3,61)):  #(np.arange(3,61)):       BLOCK HELD CONSTANT
        y_00_d = y1_00_d.iloc[:,j]    
        y_01_d = y1_01_d.iloc[:,j]
        y_10_d = y1_10_d.iloc[:,j]
        y_11_d = y1_11_d.iloc[:,j]
        y_00_u = y1_00_u.iloc[:,j]
        y_01_u = y1_01_u.iloc[:,j]
        y_10_u = y1_10_u.iloc[:,j]
        y_11_u = y1_11_u.iloc[:,j]
        y_c_00_d = np.count_nonzero(~np.isnan(y_00_d))
        if (y_c_00_d>=35):
            l_00_d, u_00_d = outlier(y_00_d)
        else:
            l_00_d, u_00_d = np.nan, np.nan      
        remove_outlier(l_00_d,u_00_d,i,j,'D','00')
        y_c_01_d = np.count_nonzero(~np.isnan(y_01_d))
        if (y_c_01_d>=35):
            l_01_d, u_01_d = outlier(y_01_d)
        else:
            l_01_d, u_01_d = np.nan, np.nan
        remove_outlier(l_01_d,u_01_d,i,j,'D','01')
        y_c_10_d = np.count_nonzero(~np.isnan(y_10_d))
        if (y_c_10_d>=35):
            l_10_d, u_10_d = outlier(y_10_d)
        else:
            l_10_d, u_10_d = np.nan, np.nan
        remove_outlier(l_10_d,u_10_d,i,j,'D','10')
        y_c_11_d = np.count_nonzero(~np.isnan(y_11_d))
        if (y_c_11_d>=35):
            l_11_d, u_11_d = outlier(y_11_d)
        else:
            l_11_d, u_11_d = np.nan, np.nan
        remove_outlier(l_11_d,u_11_d,i,j,'D','11')
        y_c_00_u = np.count_nonzero(~np.isnan(y_00_u))
        if (y_c_00_u>=35):
            l_00_u, u_00_u = outlier(y_00_u)
        else:
            l_00_u, u_00_u = np.nan, np.nan
        remove_outlier(l_00_u,u_00_u,i,j,'U','00')
        y_c_01_u = np.count_nonzero(~np.isnan(y_01_u))
        if (y_c_01_u>=35):
            l_01_u, u_01_u = outlier(y_01_u)
        else:
            l_01_u, u_01_u = np.nan, np.nan
        remove_outlier(l_01_u,u_01_u,i,j,'U','01')
        y_c_10_u = np.count_nonzero(~np.isnan(y_10_u))
        if (y_c_10_u>=35):
            l_10_u, u_10_u = outlier(y_10_u)
        else:
            l_10_u, u_10_u = np.nan, np.nan
        remove_outlier(l_10_u,u_10_u,i,j,'U','10')
        y_c_11_u = np.count_nonzero(~np.isnan(y_11_u))
        if (y_c_11_u>=35):
            l_11_u, u_11_u = outlier(y_11_u)
        else:
            l_11_u, u_11_u = np.nan, np.nan
        remove_outlier(l_11_u,u_11_u,i,j,'U','11')
               

/home/sufiyan/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
rt_00_dn.to_csv('rt_00_dn.csv')
rt_01_dn.to_csv('rt_01_dn.csv')
rt_10_dn.to_csv('rt_10_dn.csv')
rt_11_dn.to_csv('rt_11_dn.csv')
rt_00_up.to_csv('rt_00_up.csv')
rt_01_up.to_csv('rt_01_up.csv')
rt_10_up.to_csv('rt_10_up.csv')
rt_11_up.to_csv('rt_11_up.csv')